<a href="https://colab.research.google.com/github/AliciaAPerez/Death_Analytics_using_CDC_Data_with_Machine_Learning/blob/alicia/Death_Machine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import os
# Find the latest version of spark 3.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.1'
spark_version = 'spark-3.1.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:7 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:14 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:15 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic

Exception: ignored

In [ ]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

ModuleNotFoundError: ignored

In [ ]:
from pyspark import SparkFiles
from pyspark.sql.functions import col
from pyspark.sql.types import IntegerType

In [ ]:
!curl -s https://course19.fast.ai/setup/colab | bash

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#function for other data sets to be cleaned up - only can be used for 2014 and 2015 and newer, older pages use a different education marker
def data_clean_up(data):
  data = data.drop("education_1989_revision","education_reporting_flag","age_substitution_flag","age_recode_52","age_recode_27","age_recode_12", "infant_age_recode_22",
"place_of_death_and_decedents_status","method_of_disposition","autopsy", "activity_code", "place_of_injury_for_causes_w00_y34_except_y06_and_y07_", "358_cause_recode", "113_cause_recode",
"130_infant_cause_recode", "39_cause_recode", "bridged_race_flag", "race_imputation_flag", "race_recode_3", "race_recode_5", "hispanic_origin");
  data = data.filter((data.detail_age_type == 1))
  data = data.drop("detail_age_type")
  data = data.drop("number_of_entity_axis_conditions","entity_condition_4","entity_condition_5","entity_condition_6","entity_condition_7","entity_condition_8",
                                            "entity_condition_9","entity_condition_10","entity_condition_11","entity_condition_12","entity_condition_13","entity_condition_14",
                                            "entity_condition_15","entity_condition_16","entity_condition_17","entity_condition_18","entity_condition_19","entity_condition_20",
                                            "number_of_record_axis_conditions", "record_condition_1", "record_condition_2","record_condition_3","record_condition_4","record_condition_5",
                                            "record_condition_6","record_condition_7","record_condition_8","record_condition_9","record_condition_10","record_condition_11","record_condition_12",
                                            "record_condition_13","record_condition_14","record_condition_15","record_condition_16","record_condition_17","record_condition_18","record_condition_19",
                                            "record_condition_20",)
  data = data.withColumnRenamed("resident_status","resident").withColumnRenamed("education_2003_revision","education").withColumnRenamed("detail_age", "age").withColumnRenamed("current_date_year", "year").withColumnRenamed("icd_code_10th_revision", "icd_death_code").withColumnRenamed("hispanic_originrace_recode", "hispanic")
  data = data.withColumn("resident",col("resident").cast("integer")).withColumn("education",col("education").cast("integer")).withColumn("month_of_death",col("month_of_death").cast("integer")).withColumn("age",col("age").cast("integer")).withColumn("day_of_week_of_death",col("day_of_week_of_death").cast("integer")).withColumn("current_data_year",col("current_data_year").cast("integer")).withColumn("manner_of_death",col("manner_of_death").cast("integer")).withColumn("race",col("race").cast("integer")).withColumn("hispanic",col("hispanic").cast("integer"))
  data = data.fillna(value="0",subset=["entity_condition_1"]).fillna(value="0",subset=["entity_condition_2"]).fillna(value="0",subset=["entity_condition_3"])
  data = data.na.replace(["Y", "N", "U"], ["1","2","3"], "injury_at_work")
  data = data.na.replace(["M", "F"], ["1","2"], "sex")
  data = data.na.replace(["S", "M", "W", "D", "U"], ["1","2", "3", "4", "5"], "marital_status")
  data = data.select("icd_death_code","resident","education","month_of_death","sex", "age","marital_status","day_of_week_of_death","current_data_year","injury_at_work","manner_of_death","race","hispanic","entity_condition_1","entity_condition_2","entity_condition_3")
  data = data.na.drop()
  return data

In [ ]:
#function for other data sets to be cleaned up - only can be used for 2014 and 2015 and newer, older pages use a different education marker
def data_clean_up2(data):
  data = data.drop("education_2003_revision","education_reporting_flag","age_substitution_flag","age_recode_52","age_recode_27","age_recode_12", "infant_age_recode_22",
"place_of_death_and_decedents_status","method_of_disposition","autopsy", "activity_code", "place_of_injury_for_causes_w00_y34_except_y06_and_y07_", "358_cause_recode", "113_cause_recode",
"130_infant_cause_recode", "39_cause_recode", "bridged_race_flag", "race_imputation_flag", "race_recode_3", "race_recode_5", "hispanic_origin");
  data = data.filter((data.detail_age_type == 1))
  data = data.drop("detail_age_type")
  data = data.drop("number_of_entity_axis_conditions","entity_condition_4","entity_condition_5","entity_condition_6","entity_condition_7","entity_condition_8",
                                            "entity_condition_9","entity_condition_10","entity_condition_11","entity_condition_12","entity_condition_13","entity_condition_14",
                                            "entity_condition_15","entity_condition_16","entity_condition_17","entity_condition_18","entity_condition_19","entity_condition_20",
                                            "number_of_record_axis_conditions", "record_condition_1", "record_condition_2","record_condition_3","record_condition_4","record_condition_5",
                                            "record_condition_6","record_condition_7","record_condition_8","record_condition_9","record_condition_10","record_condition_11","record_condition_12",
                                            "record_condition_13","record_condition_14","record_condition_15","record_condition_16","record_condition_17","record_condition_18","record_condition_19",
                                            "record_condition_20",)
  data = data.withColumnRenamed("resident_status","resident").withColumnRenamed("education_1989_revision","education").withColumnRenamed("detail_age", "age").withColumnRenamed("current_date_year", "year").withColumnRenamed("icd_code_10th_revision", "icd_death_code").withColumnRenamed("hispanic_originrace_recode", "hispanic")
  data = data.withColumn("resident",col("resident").cast("integer")).withColumn("education",col("education").cast("integer")).withColumn("month_of_death",col("month_of_death").cast("integer")).withColumn("age",col("age").cast("integer")).withColumn("day_of_week_of_death",col("day_of_week_of_death").cast("integer")).withColumn("current_data_year",col("current_data_year").cast("integer")).withColumn("manner_of_death",col("manner_of_death").cast("integer")).withColumn("race",col("race").cast("integer")).withColumn("hispanic",col("hispanic").cast("integer"))
  data = data.fillna(value="0",subset=["entity_condition_1"]).fillna(value="0",subset=["entity_condition_2"]).fillna(value="0",subset=["entity_condition_3"])
  data = data.na.replace([0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,99], [1,1,1,1,1,1,1,1,1,2,2,2,3,4,4,4,6,7,9], "education")
  data = data.na.replace(["Y", "N", "U"], ["1","2","3"], "injury_at_work")
  data = data.na.replace(["M", "F"], ["1","2"], "sex")
  data = data.na.replace(["S", "M", "W", "D", "U"], ["1","2", "3", "4", "5"], "marital_status")
  data = data.select("icd_death_code","resident","education","month_of_death","sex", "age","marital_status","day_of_week_of_death","current_data_year","injury_at_work","manner_of_death","race","hispanic","entity_condition_1","entity_condition_2","entity_condition_3")
  data = data.na.drop()
  return data

Per previous calculations, the top 60 causes of death were used which each one affected at least 10,000 people in 2015.   
The codes are going to be simplified and renamed to smaller categories as there are mutiple that are similar (i.e. several heart conditions or cancer) and thus be slightly more broad.   
From that list, there are 16 unique codes we will use (one was unknown which we will disregard).
Less might be used if there are issues.  
I251: Heart Failure (coded as 1)  
C349: Cancer (coded as 2)  
J449: COPD (coded as 3)  
G309: Alzheimer's (coded as 4)  
F03: Dementia (coded as 5)  
J189: Pneumonia (coded as 6)  
A419: Sepsis (coded as 7)  
E149: Diabetes (coded as 8)  
G20: Parkinson's (coded as 9)  
X44: Drug exposure  (coded as 10)  
N185: Kidney Disease (coded as 11)  
K746: Cirrhosis of the Liver (coded as 12)  
W19: Falling (coded as 13)  
X74: Self-Harm (coded as 14)  
V892: Auto Accident (coded as 15)   
X95: Assault by Other (coded as 16)

In [ ]:
top_list = ["I251","C349","J449","G309","F03","J189","A419","E149","G20","X44","N185","K746","W19","X74","V892","X95"]

In [ ]:
# use this after the initial data clean up functino for the data
# code is taking the top 60 conditions and grouping them into like categories (i.e. replacing certain codes with the similar code) and then filtering only those 16 codes as listed above
# another notebook was use to figure this out
# after renaming the categories, they are filtered to only show these
def code_clean(data):
  data = data.na.replace(["C159","C169","C189","C220","C259","C509","C56","C61","C64","C679","C719","C80","C859","C900"],\
                         ["C349","C349","C349","C349","C349","C349","C349","C349","C349","C349","C349","C349","C349","C349"],"icd_death_code")
  data = data.na.replace(["E119","E142"],\
                         ["E149","E149"],"icd_death_code")
  data = data.na.replace(["F019"],\
                         ["F03"],"icd_death_code")
  data = data.na.replace(["I10","I110","I119","I120","I219","I250","I255","I129","I350","I1429","I469","I48","I500","I509","I619","I164","I694"],\
                         ["I251","I251","I251","I251","I251","I251","I251","I251","I251","I251","I251","I251","I251","I251","I251","I251","I251"],"icd_death_code")
  data = data.na.replace(["J440"],\
                         ["J449"],"icd_death_code")
  data = data.na.replace(["J690","J841"],\
                         ["J189","J189"],"icd_death_code")
  data = data.na.replace(["K703"],\
                         ["K746"],"icd_death_code")
  data = data.na.replace(["N19","N390"],\
                         ["N185","N185"],"icd_death_code")
  data = data.na.replace(["W18"],\
                         ["W19"],"icd_death_code")
  data = data.na.replace(["X42"],\
                         ["X44"],"icd_death_code")
  data = data.na.replace(["X70"],\
                         ["X74"],"icd_death_code")
  data = data.filter(data.icd_death_code.isin(top_list))
  return data

In [ ]:
# for a smaller data set, only grabbing the top 16 causes without renaming the rest of the top 60 causes to fit in these categories
# condering the top 16 has a lot of overlap in types
def code_clean_trim(data):
  data = data.filter(data.icd_death_code.isin(top_list))
  return data

In [ ]:
# UPLOAD TO DATABASE BEFORE RUNNING THIS FUNCTION
# Just personal preference that the database is slightly more clear with the code names as there are several to get renamed
def code_convert(data):
  data = data.na.replace(['I251', 'C349', 'J449', 'G309', 'F03', 'J189', 'A419', 'E149', 'G20', 'X44', 'N185', 'K746', 'W19', 'X74','V892', 'X95'],\
  ["1","2","3","4","5","6","7","8","9","10","11","12","13","14","15","16"], "icd_death_code")
  return data

In [ ]:
url ="https://aperezbucket.s3.us-east-2.amazonaws.com/2015_data.csv"
spark.sparkContext.addFile(url)
death_2015 = spark.read.csv(SparkFiles.get("2015_data.csv"), sep=",", header=True)

#run the function for clean_up and show that dataframe
death_2015_final = data_clean_up(death_2015)
# death_2015_final.show(5)

NameError: ignored

In [ ]:
death_2015_final.count()

2239254

In [ ]:
  death_2015_clean1 = death_2015_final.na.replace(["C509","C259","C189","C61","C80"],\
                         ["C349","C349","C349","C349","C349",],"icd_death_code")

In [ ]:
  death_2015_clean2 = death_2015_final.na.replace(["I219","I500","I64","I250","I119","I48"],\
                         ["I251","I251","I251","I251","I251","I251"],"icd_death_code")

In [ ]:
# ONLY USE TO HAVE SMALLER DATA SET
# USE NEXT CELL FOR MORE DATA
death_2015_clean_trim = code_clean_trim(death_2015_clean2)
# death_2015_clean_trim.count()

In [ ]:
# clean data and filter


death_2015_clean = code_clean(death_2015_final)
# death_2015_clean.show(5)   

In [ ]:
death_2015_clean.count()

748017

In [ ]:
url ="https://aperezbucket.s3.us-east-2.amazonaws.com/2014_data.csv"
spark.sparkContext.addFile(url)
death_2014 = spark.read.csv(SparkFiles.get("2014_data.csv"), sep=",", header=True)

#run the function for clean_up and show that dataframe
death_2014_final = data_clean_up(death_2014)

In [ ]:
url ="https://aperezbucket.s3.us-east-2.amazonaws.com/2013_data.csv"
spark.sparkContext.addFile(url)
death_2013 = spark.read.csv(SparkFiles.get("2013_data.csv"), sep=",", header=True)

#run the function for clean_up and show that dataframe
death_2013_final = data_clean_up2(death_2013)

In [ ]:
url ="https://aperezbucket.s3.us-east-2.amazonaws.com/2012_data.csv"
spark.sparkContext.addFile(url)
death_2012 = spark.read.csv(SparkFiles.get("2012_data.csv"), sep=",", header=True)

#run the function for clean_up and show that dataframe
death_2012_final = data_clean_up2(death_2012)

In [ ]:
url ="https://aperezbucket.s3.us-east-2.amazonaws.com/2011_data.csv"
spark.sparkContext.addFile(url)
death_2011 = spark.read.csv(SparkFiles.get("2011_data.csv"), sep=",", header=True)

#run the function for clean_up and show that dataframe
death_2011_final = data_clean_up2(death_2011)

In [ ]:
url ="https://aperezbucket.s3.us-east-2.amazonaws.com/2010_data.csv"
spark.sparkContext.addFile(url)
death_2010 = spark.read.csv(SparkFiles.get("2010_data.csv"), sep=",", header=True)

#run the function for clean_up and show that dataframe
death_2010_final = data_clean_up2(death_2010)

In [ ]:
url ="https://aperezbucket.s3.us-east-2.amazonaws.com/2009_data.csv"
spark.sparkContext.addFile(url)
death_2009 = spark.read.csv(SparkFiles.get("2009_data.csv"), sep=",", header=True)

#run the function for clean_up and show that dataframe
death_2009_final = data_clean_up2(death_2009)

In [ ]:
url ="https://aperezbucket.s3.us-east-2.amazonaws.com/2008_data.csv"
spark.sparkContext.addFile(url)
death_2008 = spark.read.csv(SparkFiles.get("2008_data.csv"), sep=",", header=True)

#run the function for clean_up and show that dataframe
death_2008_final = data_clean_up2(death_2008)

In [ ]:
url ="https://aperezbucket.s3.us-east-2.amazonaws.com/2007_data.csv"
spark.sparkContext.addFile(url)
death_2007 = spark.read.csv(SparkFiles.get("2007_data.csv"), sep=",", header=True)

#run the function for clean_up and show that dataframe
death_2007_final = data_clean_up2(death_2007)

In [ ]:
url ="https://aperezbucket.s3.us-east-2.amazonaws.com/2006_data.csv"
spark.sparkContext.addFile(url)
death_2006 = spark.read.csv(SparkFiles.get("2006_data.csv"), sep=",", header=True)

#run the function for clean_up and show that dataframe
death_2006_final = data_clean_up2(death_2006)

In [ ]:
url ="https://aperezbucket.s3.us-east-2.amazonaws.com/2005_data.csv"
spark.sparkContext.addFile(url)
death_2005 = spark.read.csv(SparkFiles.get("2005_data.csv"), sep=",", header=True)

#run the function for clean_up and show that dataframe
death_2005_final = data_clean_up2(death_2005)

In [ ]:
# COMBINE ALL DATAFRAMES
#SKIP if USING TOO MUCH MEMORY OR IF NEED SMALLER DATASET

# death1 = death_2015_clean.unionByName(death_2014_clean)
# death2 = death1.unionByName(death_2013_clean)
# death3 = death2.unionByName(death_2012_clean)
# death4 = death3.unionByName(death_2011_clean)
# death5 = death4.unionByName(death_2010_clean)
# death6 = death5.unionByName(death_2009_clean)
# death7 = death6.unionByName(death_2008_clean)
# death8 = death7.unionByName(death_2007_clean)
# death9 = death8.unionByName(death_2006_clean)
# death10 = death9.unionByName(death_2005_clean)
# death10.count()

In [ ]:
# TO GET ALL NUMBERS TO RUN PROPER MUST ANNOTATE EACH ONTO FIRST AND ADD AN ID COLUMN THEN LOAD IN SQL 
# IF USING MULTIPLE DATAFRAMES, COMBINE BEFORE DOING THIS
from pyspark.sql.functions import monotonically_increasing_id

# this is for when all the dataframes are combined
# death_numbered = death10.withColumn("id", monotonically_increasing_id() +1)

death_numbered = death_2015_clean_trim.withColumn("id", monotonically_increasing_id() +1)

# death_numbered = death_2015_clean.withColumn("id", monotonically_increasing_id() +1)

In [ ]:
death_numbered.count()

748017

In [ ]:
death_numbered.printSchema()

root
 |-- icd_death_code: string (nullable = true)
 |-- resident: integer (nullable = true)
 |-- education: integer (nullable = true)
 |-- month_of_death: integer (nullable = true)
 |-- sex: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- marital_status: string (nullable = true)
 |-- day_of_week_of_death: integer (nullable = true)
 |-- current_data_year: integer (nullable = true)
 |-- injury_at_work: string (nullable = true)
 |-- manner_of_death: integer (nullable = true)
 |-- race: integer (nullable = true)
 |-- hispanic: integer (nullable = true)
 |-- entity_condition_1: string (nullable = false)
 |-- entity_condition_2: string (nullable = false)
 |-- entity_condition_3: string (nullable = false)
 |-- id: long (nullable = false)



In [ ]:
# CONNECT TO DATABASE AND LOAD IN DATA
mode = "append"
jdbc_url="jdbc:postgresql://database-1.ck7rneirj52d.us-east-2.rds.amazonaws.com:5432/Death_Database"
config = {"user":"root", 
          "password": "postgres", 
          "driver":"org.postgresql.Driver"}

In [ ]:
death_numbered.write.jdbc(url=jdbc_url, table='cdc_death_table', mode=mode, properties=config)

In [ ]:
# now we can replace all the data with numbers 
death_final = code_convert(death_numbered)
# death_final.show(5)

In [ ]:
death_final.count()

1087358

In [ ]:
# renaming to match maching learning coding
# don't need if using Pandas
# death_machine = death_final

In [ ]:
# convert to pandas
death_machine = death_final.select("*").toPandas()
# death_machine

In [ ]:
death_machine.value_counts("icd_death_code")

icd_death_code
I251    428754
C349    126123
J449     94129
F03      92868
G309     84389
J189     36206
A419     32554
E149     25512
G20      22682
X44      19194
N185     15933
K746     15325
W19      14784
X74      12859
V892     11247
X95      11184
dtype: int64

In [ ]:
ten_list = ["I251","C349","J449","G309","F03","J189","A419","E149","G20","X44"]

In [ ]:
top_ten = death_numbered.filter(death_numbered.icd_death_code.isin(ten_list))

In [ ]:
death_machine2 = top_ten.select("*").toPandas()

In [ ]:
death_machine2.value_counts("icd_death_code")

icd_death_code
I251    428754
C349    126123
J449     94129
F03      92868
G309     84389
J189     36206
A419     32554
E149     25512
G20      22682
X44      19194
dtype: int64

In [ ]:
###MACHINE LEARNING SET UP VARIABLES
import sklearn
import sklearn.datasets
import numpy as np
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
import os
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [ ]:
wanted_data = death_machine[["education","sex", "age", "marital_status","race", "hispanic"]]
# wanted_data

In [ ]:
death_machine.head(5)

,icd_death_code,resident,education,month_of_death,sex,age,marital_status,day_of_week_of_death,current_data_year,injury_at_work,manner_of_death,race,hispanic,entity_condition_1,entity_condition_2,entity_condition_3,id
0,1,1,3,1,1,84,2,1,2015,3,7,1,6,11I500,61L031,0,1
1,2,1,6,1,1,70,2,2,2015,3,7,1,6,11I469,21R042,31C349,2
2,4,1,3,1,2,91,3,6,2015,3,7,1,6,11G309,0,0,3
3,1,1,5,1,2,89,3,5,2015,3,7,1,6,11I250,21S720,61X590,4
4,1,1,3,1,1,82,2,1,2015,3,7,1,6,11I500,0,0,5


In [ ]:
# For use with Pyspark not Pandas
# data_array =  np.array(death_machine.select("resident", "education", "sex", "age", "marital_status", "injury_at_work", "manner_of_death", "race", "hispanic").collect())
# data_array =  np.array(death_machine.select("education", "sex", "age", "marital_status", "injury_at_work", "manner_of_death", "race", "hispanic").collect())
# data_array =  np.array(death_machine.select("education","sex", "age", "marital_status","manner_of_death", "race", "hispanic").collect())
# data_array =  np.array(death_machine.select("sex", "age", "marital_status","race", "hispanic").collect())


data_array =  np.array(death_machine.select("education","sex", "age", "marital_status","race", "hispanic").collect())
y = death_machine.select("icd_death_code")

In [ ]:
X = data_array[:, :-1]
y = data_array[:,-1]

In [ ]:
X = X.astype(str)
# reshape target to be a 2d array
y = y.reshape((len(y), 1))

In [ ]:
print(X.shape, y.shape)

In [ ]:
X = wanted_data
y = death_machine["icd_death_code"]
print(X.shape, y.shape)

(1043743, 6) (1043743,)


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [ ]:
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(782807, 6) (782807,)
(260936, 6) (260936,)


In [ ]:
X_train = X_train.astype(int)
X_test = X_test.astype(int)
y_train = y_train.astype(int)
y_test = y_test.astype(int)

In [ ]:
X_scaler = MinMaxScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [ ]:
print(X_train_scaled.shape)
print(X_test_scaled.shape)
print(y_train.shape)
print(y_test.shape)

(782807, 6)
(260936, 6)
(782807,)
(260936,)


In [ ]:
# from tensorflow.keras.utils import to_categorical
# label_encoder = LabelEncoder()
# label_encoder.fit(y_train)
# encoded_y_train = label_encoder.transform(y_train)
# encoded_y_test = label_encoder.transform(y_test)

In [ ]:
# y_train_categorical = to_categorical(encoded_y_train)
# y_test_categorical = to_categorical(encoded_y_test)

In [ ]:
# y_train_categorical.shape

In [ ]:
model9 = Sequential()

In [ ]:
model9.add(Dense(units=100, activation='relu', input_dim=6))
model9.add(Dense(units=100, activation='relu'))
model9.add(Dense(units=1, activation='softmax'))

In [ ]:
model9.compile(loss="categorical_crossentropy", optimizer="adam", metrics=['accuracy'])

In [ ]:
model9.fit(
    X_train_scaled,
    y_train,
    epochs=30,
    shuffle=True,
    verbose=2
)

Epoch 1/30
24463/24463 - 33s - loss: 0.0000e+00 - accuracy: 0.4111
Epoch 2/30
24463/24463 - 31s - loss: 0.0000e+00 - accuracy: 0.4111
Epoch 3/30
24463/24463 - 32s - loss: 0.0000e+00 - accuracy: 0.4111
Epoch 4/30
24463/24463 - 32s - loss: 0.0000e+00 - accuracy: 0.4111
Epoch 5/30
24463/24463 - 31s - loss: 0.0000e+00 - accuracy: 0.4111
Epoch 6/30
24463/24463 - 31s - loss: 0.0000e+00 - accuracy: 0.4111
Epoch 7/30
24463/24463 - 31s - loss: 0.0000e+00 - accuracy: 0.4111
Epoch 8/30
24463/24463 - 31s - loss: 0.0000e+00 - accuracy: 0.4111
Epoch 9/30
24463/24463 - 31s - loss: 0.0000e+00 - accuracy: 0.4111
Epoch 10/30
24463/24463 - 31s - loss: 0.0000e+00 - accuracy: 0.4111
Epoch 11/30
24463/24463 - 31s - loss: 0.0000e+00 - accuracy: 0.4111
Epoch 12/30
24463/24463 - 32s - loss: 0.0000e+00 - accuracy: 0.4111
Epoch 13/30
24463/24463 - 31s - loss: 0.0000e+00 - accuracy: 0.4111
Epoch 14/30
24463/24463 - 31s - loss: 0.0000e+00 - accuracy: 0.4111
Epoch 15/30
24463/24463 - 31s - loss: 0.0000e+00 - accura

In [ ]:
model9.save("Death_machine_keras9.h5")

In [ ]:
len(model9.weights)
# print(model.best_score_)
model9.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_12 (Dense)             (None, 100)               700       
_________________________________________________________________
dense_13 (Dense)             (None, 100)               10100     
_________________________________________________________________
dense_14 (Dense)             (None, 1)                 101       
Total params: 10,901
Trainable params: 10,901
Non-trainable params: 0
_________________________________________________________________


In [ ]:
from tensorflow.keras.models import load_model
model_9 = load_model("Death_machine_keras9.h5")

In [ ]:
model_loss, model_accuracy = model_9.evaluate(X_test_scaled, y_test, verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

8155/8155 - 6s - loss: 0.0000e+00 - accuracy: 0.4099
Loss: 0.0, Accuracy: 0.40987443923950195


In [ ]:
encoded_predictions = np.argmax(model9.predict(X_test_scaled[:10]), axis=-1)

In [ ]:
print(f"Predicted classes: {encoded_predictions}")
print(f"Actual Labels: {list(y_test[:10])}")

Predicted classes: [0 0 0 0 0 0 0 0 0 0]
Actual Labels: [14, 5, 5, 1, 1, 1, 5, 2, 4, 2]


In [ ]:
death_machine2.head(5)

,icd_death_code,resident,education,month_of_death,sex,age,marital_status,day_of_week_of_death,current_data_year,injury_at_work,manner_of_death,race,hispanic,entity_condition_1,entity_condition_2,entity_condition_3,id
0,I251,1,3,1,1,84,2,1,2015,3,7,1,6,11I500,61L031,0,1
1,C349,1,6,1,1,70,2,2,2015,3,7,1,6,11I469,21R042,31C349,2
2,G309,1,3,1,2,91,3,6,2015,3,7,1,6,11G309,0,0,3
3,I251,1,5,1,2,89,3,5,2015,3,7,1,6,11I250,21S720,61X590,4
4,I251,1,3,1,1,82,2,1,2015,3,7,1,6,11I500,0,0,5


In [ ]:
wanted_data = death_machine2[["education","sex", "age", "marital_status","race", "hispanic"]]

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [ ]:
X = wanted_data
y = death_machine2["icd_death_code"]
print(X.shape, y.shape)

(962411, 6) (962411,)


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [ ]:
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(721808, 6) (721808,)
(240603, 6) (240603,)


In [ ]:
X_train = X_train.astype(int)
X_test = X_test.astype(int)

In [ ]:
X_scaler = MinMaxScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [ ]:
print(X_train_scaled.shape)
print(X_test_scaled.shape)
print(y_train.shape)
print(y_test.shape)

(721808, 6)
(240603, 6)
(721808,)
(240603,)


In [ ]:
from tensorflow.keras.utils import to_categorical
label_encoder = LabelEncoder()
label_encoder.fit(y_train)
encoded_y_train = label_encoder.transform(y_train)
encoded_y_test = label_encoder.transform(y_test)

In [ ]:
y_train_categorical = to_categorical(encoded_y_train)
y_test_categorical = to_categorical(encoded_y_test)

In [ ]:
y_train_categorical.shape

(721808, 10)

In [ ]:
model10 = Sequential()

In [ ]:
model10.add(Dense(units=100, activation='relu', input_dim=6))
model10.add(Dense(units=100, activation='relu'))
model10.add(Dense(units=10, activation='softmax'))

In [ ]:
# Compile and fit the model
model10.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
model10.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_15 (Dense)             (None, 100)               700       
_________________________________________________________________
dense_16 (Dense)             (None, 100)               10100     
_________________________________________________________________
dense_17 (Dense)             (None, 10)                1010      
Total params: 11,810
Trainable params: 11,810
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model10.fit(
    X_train_scaled,
    y_train_categorical,
    epochs=30,
    shuffle=True,
    verbose=2
)

Epoch 1/30
22557/22557 - 29s - loss: 1.6425 - accuracy: 0.4501
Epoch 2/30
22557/22557 - 29s - loss: 1.6264 - accuracy: 0.4511
Epoch 3/30
22557/22557 - 29s - loss: 1.6238 - accuracy: 0.4511
Epoch 4/30
22557/22557 - 29s - loss: 1.6225 - accuracy: 0.4513
Epoch 5/30
22557/22557 - 30s - loss: 1.6219 - accuracy: 0.4512
Epoch 6/30
22557/22557 - 30s - loss: 1.6211 - accuracy: 0.4514
Epoch 7/30
22557/22557 - 29s - loss: 1.6208 - accuracy: 0.4514
Epoch 8/30
22557/22557 - 31s - loss: 1.6205 - accuracy: 0.4516
Epoch 9/30
22557/22557 - 37s - loss: 1.6201 - accuracy: 0.4516
Epoch 10/30
22557/22557 - 29s - loss: 1.6198 - accuracy: 0.4515
Epoch 11/30
22557/22557 - 28s - loss: 1.6196 - accuracy: 0.4516
Epoch 12/30
22557/22557 - 29s - loss: 1.6193 - accuracy: 0.4517
Epoch 13/30
22557/22557 - 28s - loss: 1.6191 - accuracy: 0.4516
Epoch 14/30
22557/22557 - 29s - loss: 1.6189 - accuracy: 0.4517
Epoch 15/30
22557/22557 - 29s - loss: 1.6189 - accuracy: 0.4517
Epoch 16/30
22557/22557 - 30s - loss: 1.6190 - ac

In [ ]:
model_loss, model_accuracy = model10.evaluate(
    X_test_scaled, y_test_categorical, verbose=2)
print(
    f"Normal Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}")

7519/7519 - 6s - loss: 1.6271 - accuracy: 0.4509
Normal Neural Network - Loss: 1.6270573139190674, Accuracy: 0.45093369483947754


In [ ]:
encoded_predictions = np.argmax(model10.predict(X_test_scaled[:5]), axis=-1)
prediction_labels = label_encoder.inverse_transform(encoded_predictions)

In [ ]:
print(f"Predicted classes: {prediction_labels}")
print(f"Actual Labels: {list(y_test[:5])}")

Predicted classes: ['C349' 'I251' 'I251' 'I251' 'I251']
Actual Labels: ['G20', 'G309', 'I251', 'I251', 'I251']


In [ ]:
model10.save("Death_machine_keras10.h5")

In [ ]:
from tensorflow.keras.models import load_model
model_10 = load_model("Death_machine_keras10.h5")

OSError: ignored

In [ ]:
model_loss, model_accuracy = model_10.evaluate(X_test_scaled, y_test_categorical, verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

7519/7519 - 6s - loss: 1.6271 - accuracy: 0.4509
Loss: 1.6270573139190674, Accuracy: 0.45093369483947754


In [ ]:
model11 = Sequential()

NameError: ignored

In [ ]:
model11.add(Dense(units=100, activation='relu', input_dim=6))
model11.add(Dense(units=100, activation='relu'))
model11.add(Dense(units=100, activation='relu'))
model11.add(Dense(units=100, activation='relu'))
model11.add(Dense(units=100, activation='relu'))
model11.add(Dense(units=10, activation='softmax'))

NameError: ignored

In [ ]:
# Compile and fit the model
model11.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
model11.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_21 (Dense)             (None, 100)               700       
_________________________________________________________________
dense_22 (Dense)             (None, 100)               10100     
_________________________________________________________________
dense_23 (Dense)             (None, 100)               10100     
_________________________________________________________________
dense_24 (Dense)             (None, 100)               10100     
_________________________________________________________________
dense_25 (Dense)             (None, 100)               10100     
_________________________________________________________________
dense_26 (Dense)             (None, 10)                1010      
Total params: 42,110
Trainable params: 42,110
Non-trainable params: 0
__________________________________________________

In [ ]:
model11.fit(
    X_train_scaled,
    y_train_categorical,
    epochs=30,
    shuffle=True,
    verbose=2
)

Epoch 1/30
22557/22557 - 46s - loss: 1.6507 - accuracy: 0.4494
Epoch 2/30
22557/22557 - 44s - loss: 1.6309 - accuracy: 0.4508
Epoch 3/30
22557/22557 - 44s - loss: 1.6270 - accuracy: 0.4511
Epoch 4/30
22557/22557 - 45s - loss: 1.6250 - accuracy: 0.4512
Epoch 5/30
22557/22557 - 45s - loss: 1.6241 - accuracy: 0.4514
Epoch 6/30
22557/22557 - 46s - loss: 1.6233 - accuracy: 0.4513
Epoch 7/30
22557/22557 - 45s - loss: 1.6228 - accuracy: 0.4514
Epoch 8/30
22557/22557 - 44s - loss: 1.6222 - accuracy: 0.4514
Epoch 9/30
22557/22557 - 45s - loss: 1.6219 - accuracy: 0.4514
Epoch 10/30
22557/22557 - 44s - loss: 1.6215 - accuracy: 0.4514
Epoch 11/30
22557/22557 - 45s - loss: 1.6212 - accuracy: 0.4517
Epoch 12/30
22557/22557 - 45s - loss: 1.6209 - accuracy: 0.4515
Epoch 13/30
22557/22557 - 45s - loss: 1.6210 - accuracy: 0.4515
Epoch 14/30
22557/22557 - 44s - loss: 1.6206 - accuracy: 0.4516
Epoch 15/30
22557/22557 - 44s - loss: 1.6207 - accuracy: 0.4517
Epoch 16/30
22557/22557 - 44s - loss: 1.6203 - ac

In [ ]:
test = [1,2,3,4,5,6]
model11.predict(test)

NameError: ignored

In [ ]:
# import pickle
# pickle.dump(model11, open('model_death.pkl', 'wb'))
# model11_load = pickle.load(open('model_death_pkl.pkl', 'rb'))
# print(model11_load)

# serialize model to JSON
model_json = model11.to_json()
with open("model_death11.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model11.save_weights("model_death11.h5")
print("Saved model to disk")


Saved model to disk


In [ ]:

# load json and create model
json_file = open('model_death11.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("model_death11.h5")
print("Loaded model from disk")
 
# evaluate loaded model on test data
loaded_model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
score = loaded_model.evaluate(X, Y, verbose=0)
print("%s: %.2f%%" % (loaded_model.metrics_names[1], score[1]*100))

NameError: ignored